In [ ]:
# Import required libraries and dependencies
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame and make the index the "coin_id" column.
market_data_df = pd.read_csv("Resources/crypto_market_data.csv", index_col="coin_id" )

# Display sample data
market_data_df.head(10)

In [ ]:
# Generate summary statistics
market_data_df.describe()

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled = StandardScaler().fit_transform(market_data_df[["price_change_percentage_24h", 
                                     "price_change_percentage_7d", "price_change_percentage_14d",	
                                   "price_change_percentage_30d", "price_change_percentage_60d",	
                                   "price_change_percentage_200d",	"price_change_percentage_1y"]])

market_scaled

In [ ]:
# Create a DataFrame with the scaled data
market_transformed = pd.DataFrame (market_scaled, columns=["price_change_percentage_24h", 
                                     "price_change_percentage_7d", "price_change_percentage_14d",	
                                   "price_change_percentage_30d", "price_change_percentage_60d",	
                                   "price_change_percentage_200d",	"price_change_percentage_1y"]
)

# Copy the crypto names from the original data
market_data_df = market_data_df.reset_index()
market_transformed["coin_id"] = market_data_df["coin_id"]

# Set the coinid column as index

market_transformed = market_transformed.set_index("coin_id")

# Display sample data
market_transformed.head()

### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

# Create an empty list to store the inertia values
inertia =[]
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using the scaled DataFrame
# 3. Append the model.inertia_ to the inertia list
for i in k: 
    k_model = KMeans(n_clusters=i, n_init='auto', random_state=1)
    k_model.fit(market_scaled)
    inertia.append(k_model.inertia_)

# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)
# Display the DataFrame
df_elbow

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.plot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve",
    xticks = k
)

#### Answer the following question: 
**Question:** What is the best value for `k`?

**Answer:** 4 is the best value for k

### Cluster Cryptocurrencies with K-means Using the Original Scaled Data.

In [ ]:
# Initialize the K-Means model using the best value for k
model =KMeans(n_clusters=4, n_init='auto', random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(market_transformed)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_lower = model.predict(market_transformed)

# View the resulting array of cluster values.
k_lower

In [ ]:
# Create a copy of the DataFrame
market_predictions = market_transformed.copy()


In [ ]:
# Add a new column to the DataFrame with the predicted clusters
market_predictions['clusters_lower'] = k_lower 
# Display sample data
market_predictions.head()

In [ ]:
# Create a scatter plot using Pandas plot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Use "rainbow" for the color to better visualize the data.

market_predictions.plot.scatter(
     x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c= "clusters_lower",
    colormap = "coolwarm"

)

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` on the original scaled DataFrame to reduce to three principal components.
pca_info = pca.fit_transform(market_scaled)

# View the first five rows of the DataFrame. 
pca_info[:5]

In [ ]:
# Retrieve the explained variance to determine how much information  can be attributed to each principal component.
pca.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [ ]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data
pca_info_df = pd.DataFrame(pca_info, columns=['PCA1','PCA2','PCA3']
)
# Copy the crypto names from the original data
pca_info_df["coin_id"] = market_data_df["coin_id"]
# Set the coinid column as index
pca_info_df= pca_info_df.set_index("coin_id")
# Display sample data
pca_info_df.head()

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list (range(1,11))

# Create an empty list to store the inertia values
inertia=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using PCA DataFrame.
# 3. Append the model.inertia_ to the inertia list
for i in k: 
    k_model = KMeans (n_clusters=i, n_init='auto', random_state = 1)
    k_model.fit(pca_info_df)
    inertia.append (k_model.inertia_)

# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create a DataFrame with the data to plot the Elbow curve


# Display the DataFrame
df_elbow.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.plot.line (
      x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k

)

#### Answer the following questions: 
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it actually confirms that 4, is the best vaule

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model =KMeans(n_clusters=4, n_init='auto', random_state=1)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(pca_info_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
preditions_k4 = model.predict (pca_info_df)

# Print the resulting array of cluster values.
preditions_k4

In [ ]:
# Create a copy of the DataFrame with the PCA data
preditctions_k4_df = pca_info_df.copy()

# Add a new column to the DataFrame with the predicted clusters
preditctions_k4_df ["Predicted Clusters"] = preditions_k4

# Display sample data
preditctions_k4_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting `x="PCA1"` and `y="PCA2"`. 
preditctions_k4_df.plot.scatter(
    x="PCA1",
    y="PCA2",
    c="Predicted Clusters",
    colormap = 'winter'   
)



### Determine the Weights of Each Feature on each Principal Component

In [ ]:
# Use the columns from the original scaled DataFrame as the index.
pca_component_weights = pd.DataFrame(pca.components_.T, columns=['PCA1', 'PCA2','PCA3'], index=market_transformed.columns)
pca_component_weights

#### Answer the following question: 

* **Question:** Which features have the strongest positive or negative influence on each component? 
 
* **Answer:** 
    

Change in price during 24hrs had the major negative (41%) influence on PCA on the contrary 1 year was the one positive impact 56%. 

However for PCA2, 1 year was the one that had negative impact 15%.

For PC3 the 60days had the more negative impact with 36%. 


 